Step 1: Set Up Your Django Project

In [1]:
# Create a new Django project
django-admin startproject consultancy_website
cd consultancy_website

# Create a new app
python manage.py startapp consultancy


SyntaxError: invalid syntax (1454254681.py, line 2)

Step 2: Update Settings

In [ ]:
# consultancy_website/settings.py

INSTALLED_APPS = [
    ...,
    'consultancy',
    'django.contrib.sites',  # For site-specific permissions
]


Step 3: Create User and Role Models

In [ ]:
# consultancy/models.py
from django.db import models
from django.contrib.auth.models import AbstractUser

class Role(models.Model):
    name = models.CharField(max_length=100, unique=True)

    def __str__(self):
        return self.name

class CustomUser(AbstractUser):
    role = models.ForeignKey(Role, on_delete=models.SET_NULL, null=True)

    def __str__(self):
        return self.username


Step 4: Configure the Custom User Model

In [ ]:
# consultancy_website/settings.py

AUTH_USER_MODEL = 'consultancy.CustomUser'


Step 5: Create Database Tables

In [ ]:
python manage.py makemigrations
python manage.py migrate


Step 6: Admin Interface

In [ ]:
# consultancy/admin.py
from django.contrib import admin
from .models import Role, CustomUser

admin.site.register(Role)
admin.site.register(CustomUser)


Step 7: Create Views with Role-Based Access

In [ ]:
# consultancy/views.py
from django.shortcuts import render
from django.contrib.auth.decorators import login_required
from django.contrib.auth.mixins import LoginRequiredMixin
from django.views import View
from django.http import HttpResponseForbidden

class RoleRequiredMixin(LoginRequiredMixin):
    def dispatch(self, request, *args, **kwargs):
        if request.user.role.name != self.required_role:
            return HttpResponseForbidden()
        return super().dispatch(request, *args, **kwargs)

class AdminDashboardView(RoleRequiredMixin, View):
    required_role = 'Admin'

    def get(self, request):
        return render(request, 'consultancy/admin_dashboard.html')

class UserDashboardView(RoleRequiredMixin, View):
    required_role = 'User'

    def get(self, request):
        return render(request, 'consultancy/user_dashboard.html')


Step 8: Create URLs

In [ ]:
# consultancy/urls.py
from django.urls import path
from .views import AdminDashboardView, UserDashboardView

urlpatterns = [
    path('admin/', AdminDashboardView.as_view(), name='admin_dashboard'),
    path('user/', UserDashboardView.as_view(), name='user_dashboard'),
]


In [ ]:
# consultancy_website/urls.py
from django.contrib import admin
from django.urls import path, include

urlpatterns = [
    path('admin/', admin.site.urls),
    path('', include('consultancy.urls')),
]
